
# Predicting House Price Amount

---
**Author** : Rahul Bordoloi

**Email** : <rahulbordoloi24@gmail.com>, <rahul.bordoloi@highradius.com>

---



In [4]:
# Supress all Warnings
import warnings
warnings.filterwarnings('ignore')

In [23]:
# Check Core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.18.0


In [7]:
# Setting up a Workspace [Creating a New One if you don't have one]
from azureml.core import Workspace
ws = Workspace.create(name = 'rony_ws',
                     subscription_id = '69feeb77-e26a-4eda-a7a1-0c3a1d3c1e31',
                     resource_group = 'rony-ml',
                     create_resource_group = False,
                     location = 'eastus')

# Or Directing Importing a Pre-Existing Workspace from a JSON File if you've one
## ws = Workspace.from_config(path = "config.json")

Deploying StorageAccount with name ronywsstorage76a1011e93f.
Deploying KeyVault with name ronywskeyvault9e252d41de.
Deploying AppInsights with name ronywsinsights29eec5f811.
Deployed AppInsights with name ronywsinsights29eec5f811. Took 6.53 seconds.
Deployed KeyVault with name ronywskeyvault9e252d41de. Took 23.22 seconds.
Deployed StorageAccount with name ronywsstorage76a1011e93f. Took 31.07 seconds.
Deploying Workspace with name rony_ws.
Deployed Workspace with name rony_ws. Took 45.97 seconds.


In [8]:
%%writefile my_azure_demo.py

# Importing Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (15, 5)
import os

# Loading a Dataset [Boston Dataset]
from sklearn.datasets import load_boston
data = load_boston()

# Train-Test Data Split
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_est = train_test_split(data.data[:, 5].reshape(-1, 1), data.target.reshape(-1, 1))

# Creating a Linear Regression Model
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(x_train, y_train)

Writing my_azure_demo.py


In [17]:
# Saving the Model
import joblib
os.makedirs('output', exist_ok = True)
joblib.dump(value = model, filename = 'output/model_lr.pkl')

['output/model_lr.pkl']

In [20]:
# Registering Model into Workspace
from azureml.core.model import Model
azure_model = Model.register(workspace = ws,
                            model_path = 'output/model_lr.pkl',
                            model_name = 'boston_lr')

Registering model boston_lr


<b> Training a Model using Azure ML

In [21]:
# Checking out the VMs Available
from azureml.core.runconfig import RunConfiguration
from azureml.core.compute import AmlCompute
list_vms = AmlCompute.supported_vmsizes(workspace = ws)
print(list_vms)

[{'name': 'Standard_DS1_v2', 'vCPUs': 1, 'gpus': 0, 'memoryGB': 3.5, 'maxResourceVolumeMB': 7168}, {'name': 'Standard_DS2_v2', 'vCPUs': 2, 'gpus': 0, 'memoryGB': 7.0, 'maxResourceVolumeMB': 14336}, {'name': 'Standard_DS3_v2', 'vCPUs': 4, 'gpus': 0, 'memoryGB': 14.0, 'maxResourceVolumeMB': 28672}, {'name': 'Standard_DS4_v2', 'vCPUs': 8, 'gpus': 0, 'memoryGB': 28.0, 'maxResourceVolumeMB': 57344}, {'name': 'Standard_DS5_v2', 'vCPUs': 16, 'gpus': 0, 'memoryGB': 56.0, 'maxResourceVolumeMB': 114688}, {'name': 'Standard_DS11_v2', 'vCPUs': 2, 'gpus': 0, 'memoryGB': 14.0, 'maxResourceVolumeMB': 28672}, {'name': 'Standard_DS12_v2', 'vCPUs': 4, 'gpus': 0, 'memoryGB': 28.0, 'maxResourceVolumeMB': 57344}, {'name': 'Standard_DS13_v2', 'vCPUs': 8, 'gpus': 0, 'memoryGB': 56.0, 'maxResourceVolumeMB': 114688}, {'name': 'Standard_DS14_v2', 'vCPUs': 16, 'gpus': 0, 'memoryGB': 112.0, 'maxResourceVolumeMB': 229376}, {'name': 'Standard_DS15_v2', 'vCPUs': 20, 'gpus': 0, 'memoryGB': 140.0, 'maxResourceVolumeMB

In [22]:
# We shall simple just use Standard_DS1_v2 [as it's Free :p]
compute_config = RunConfiguration()
compute_config.target = "amlcompute"
compute_config.amlcompute.vm_size = "Standard_DS1_v2"


                        DEPRECATED
                        We will be deprecating the run based creation of compute in the next release.
                        We recommend creating an actual Amlcompute cluster as a persistent compute target, 
                        and using the cluster name as the compute target in your run configuration. 
                        See example notebook here: aka.ms/amlcomputenb
                        


In [30]:
# Exporting Dependencies
from azureml.core.conda_dependencies import CondaDependencies
import sklearn, matplotlib
dependencies = CondaDependencies()
dependencies.set_python_version("3.6.8")
dependencies.add_pip_package(f"scikit-learn=={sklearn.__version__}")
dependencies.add_pip_package(f"numpy=={np.__version__}")
dependencies.add_pip_package(f"pandas=={pd.__version__}")
dependencies.add_pip_package(f"matplotlib=={matplotlib.__version__}")
compute_config.environment.python.conda_dependencies = dependencies

<b> Training Model in the Cloud

In [36]:
# Training our Model in the Cloud
from azureml.core.experiment import Experiment
from azureml.core import ScriptRunConfig
script_run_config = ScriptRunConfig(source_directory = ".", script = "my_azure_demo.py", run_config = compute_config)
experiment = Experiment(workspace = ws, name = "boston_lr_model")
run = experiment.submit(config = script_run_config)
run.wait_for_completion(show_output = True)

<b> Deploying our Model in the Cloud

In [41]:
# Finding our the File Names Associated with our Workspace
run.get_file_names()

['my_azure_demo.py', 'boston_lr_model.pkl']


In [37]:
# Downloading our Model
run.download_file(name = "output/azure_cloud.pkl")

[output/azure_cloud.pkl]


In [38]:
# Loading the Model into Local Env
import joblib
run_model = joblib.load('output/azure_cloud.pkl')

In [40]:
# If we're not Satisfied, we can retrain the Model Again
from azureml.core.model import Model
model = Model.register(workspace = ws, model_path = "output/model_lr.pkl", model_name = "cloud_boston_lr")

Registering model cloud_boston_lr


# End